In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import scripts.implementations as lib  # Add personal library
import scripts.proj1_helpers as helper  # Add personal library

%matplotlib inline
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4)

DATA_FOLDER = 'data'
DATA_TRAIN = os.path.join(DATA_FOLDER, 'train.csv')
DATA_TEST = os.path.join(DATA_FOLDER, 'test.csv')


# 1. Data Cleaning 


## 1.1 Data Exploration

We first load the data to see what are the repartition of the data. In our case prediction gives `s` for signal and `b` for backgroud. In this case around 2/3 of the data (65.73%) are labeled as background.

In [ ]:
y_train, x_train, ids, header = helper.load_csv_data(DATA_TRAIN)

In [ ]:
print('Repartition of {} labels, s: {:.2f}%, b: {:.2f}%'.format(
    len(y_train), np.mean(y_train==1)*100, np.mean(y_train==-1)*100))

According to [the Higgs boson machine learning challenge](https://higgsml.lal.in2p3.fr/files/2014/04/documentation_v1.8.pdf) some variable are indicated as "may be undefined" when it can happen that they are meaning-
less or cannot be computed. In this case, their value is -999.0, which is outside the normal range of all variables. Let's put them to NaN so they will be easier to handle.

In [ ]:
x_train[x_train == -999] = np.nan

Let's now take a look at the repartition of the NaN along the features. We can see that some features seems to have the same amount of NaN value. The second graph shows that some features seems to have NaNs values axactly at the same location.

In [ ]:
plt.figure(figsize=(16,4))
plt.bar(np.arange(len(header)), np.sum(np.isnan(x_train), axis=0))
plt.xticks(np.arange(len(header)), header, rotation='vertical')
plt.ylim(0, len(y_train)); plt.xlabel('Features'); plt.xlabel('#Sample'); plt.title('NaN sum per feature')
plt.grid(); plt.show();

In [ ]:
plt.figure(figsize=(14, 20))
plt.matshow(np.isnan(x_train)[:100, :].T)
plt.yticks(np.arange(len(header)), header)
plt.xlabel('Features'); plt.xlabel('#Sample'); plt.title('NAN sum per feature')
plt.show(); 

Does the NaN value gave us any information (`s` or `b`) ? We can see that is NaN is not present we are more likely to find a signal `s`. If NaN is present it seems that we are close to the initial distribution with 34%-66% ratio.

In [ ]:
print('NaN is present, s: {:.2f}, b: {:.2f}'.format(
    np.mean(y_train[np.any(np.isnan(x_train), axis=1)] == 1), 
    np.mean(y_train[np.any(np.isnan(x_train), axis=1)] == -1)))
print('NaN is not present, s: {:.2f}, b: {:.2f}'.format(
    np.mean(y_train[~np.any(np.isnan(x_train), axis=1)] == 1), 
    np.mean(y_train[~np.any(np.isnan(x_train), axis=1)] == -1)))

We can also take a look at the feature ranges. it can give us insights of the data. We can see that features (16), (19), (21), (26) and (29) are actually angles (range in $[-\pi, \pi]$). To be certain we checked it directly on the documentation. Note that we are ignoring the NaN values to compute the min and max.

We have to be careful with those results since the output gives us no imformation about the data distribution!

In [ ]:
for i, feature in enumerate(x_train.T):
    print('Feature {} - {} has range: [{:.4f}, {:.4f}]'.format(
        i+1, header[i], np.nanmin(feature), np.nanmax(feature)))

## 1.2 Data Cleaning - Naive

One solution to deal with NaN is to remove feature vectors that contains NaN values. After this we can normalize the data (0 mean and 1 standard deviation).

In [ ]:
# Remove features with NaN
keep_id = np.nonzero(np.sum(np.isnan(x_train), axis=0) == 0)[0]
x_naive = x_train[:, keep_id]
# normalize features
x_naive = (x_naive - np.mean(x_naive, axis=0))/np.std(x_naive, axis=0)

In [ ]:
from scripts.ml import cross_validation_ls

degrees = np.linspace(1, 6, 6).astype(int)
for i, degree in enumerate(degrees):
    acc, _, _ = cross_validation_ls(y_train, x_naive, degree=degree)
    print('{}/{} Least square deg {} with acc {:.4f}'.format(i+1, len(degrees), degree, acc))

## 1.3 Data Cleaning - Dealing with NaN


In [ ]:
# normalize features
x_no_nan = x_train.copy()
x_no_nan = (x_no_nan - np.nanmean(x_no_nan, axis=0))/np.nanstd(x_no_nan, axis=0)
x_no_nan = np.nan_to_num(x_no_nan)
print('\nStd:', np.std(x_no_nan, axis=0))

In [ ]:
for i, degree in enumerate(degrees):
    acc, _, _ = cross_validation_ls(y_train, x_no_nan, degree=degree)
    print('{}/{} Least square deg {} with acc {:.4f}'.format(i+1, len(degrees), degree, acc))

## 1.4 Data Cleaning - Dealing with physics

In [ ]:
tags, count = np.unique(x_train[:, 22], return_counts=True)
print('Repartition of #jet {} along data {}%'.format(tags, 100*count/len(y_train)))

In [ ]:
plt.figure(figsize=(14, 4))
nan_j = []
for i in range(4):
    nan_j.append(np.array([ np.sum(np.logical_and(x_train[:, 22] == i, np.isnan(f))) for f in x_train.T]))
    plt.bar(np.arange(len(header)), nan_j[-1], 
            label='#Jet={}'.format(i), bottom = np.sum(np.array(nan_j[:-1]), axis=0))
plt.xticks(np.arange(len(header)), header, rotation='vertical'); 
plt.ylim(0, len(y_train));
plt.xlabel('Features'); plt.xlabel('#Sample'); plt.title('NaN sum per feature and Jet number')
plt.legend(); plt.grid(); plt.show()

In [ ]:
def get_data_jet(y, x, id_current_jet, id_jet=22):
    id_select = (x[:, id_jet] == id_current_jet)
    x_jet = x[id_select, :]
    y_jet = y[id_select]
    return id_select, y_jet, x_jet[:, np.logical_and(~np.any(np.isnan(x_jet), axis=0), np.arange(x.shape[1]) != id_jet)]
   
models = []
for i in range(4):
    _, y_train_j, x_train_j = get_data_jet(y_train, x_train, i)
    # Last vector is only 0's for #jet == 0, we remove it
    if i != 0:
        x_train_j = (x_train_j - np.mean(x_train_j, axis=0))/np.std(x_train_j, axis=0)
    else:
        x_train_j = (x_train_j[:, :-1] - np.mean(x_train_j[:, :-1], axis=0))/np.std(x_train_j[:, :-1], axis=0)
    models.append( {'y_train': y_train_j, 'x_train': x_train_j} )

In [ ]:
degrees = np.linspace(1, 12, 12).astype(int)
best_acc = np.zeros(len(models))
for j, model in enumerate(models):
    acc = np.zeros(len(degrees))
    tr = np.zeros(len(degrees))
    te = np.zeros(len(degrees))
    for i, degree in enumerate(degrees):
        acc[i], tr[i], te[i] = cross_validation_ls(model['y_train'], model['x_train'], degree=degree)
        print('Jet-{}-{}: Least square max acc: {:.4f}, {:.4f}, {:.4f}'.format(
            j, degree, acc[i], tr[i], te[i]))
    best_acc[j] = np.max(acc)
    #print('Jet-{}: Least square max acc: {:.4f}, {:.4f}, {:.4f}, for deg: {}'.format(
    #    j, np.max(acc), tr[np.argmax(acc)], te[np.argmax(acc)], degrees[np.argmax(acc)]))
# Sum accuracy with the actual repartition of #jet over dataset
print('\n-> Theorical overal accuracy: {:.4f}'.format(np.sum(best_acc * count/len(y_train))))

# 2. Submission test

In [ ]:
y_test, x_test, ids, header = helper.load_csv_data(DATA_TEST)
x_test[x_test == -999] = np.nan

In [ ]:
sum_ = 0
for i in range(4):
    id_test_j, y_test_j, x_test_j = get_data_jet(y_test, x_test, i)
    # Last vector is only 0's for #jet == 0, we remove it
    if i != 0:
        x_test_j = (x_test_j - np.mean(x_test_j, axis=0))/np.std(x_test_j, axis=0)
    else:
        x_test_j = (x_test_j[:, :-1] - np.mean(x_test_j[:, :-1], axis=0))/np.std(x_test_j[:, :-1], axis=0)
    sum_ += np.sum(id_test_j)
    models[i]['id_test'] = id_test_j 
    models[i]['x_test'] = x_test_j
    
print(sum_, len(y_test))
print(np.nonzero(models[3]['id_test']))
print(np.nonzero(x_test[:, 22]==3))

In [ ]:
from scripts.implementations import build_poly, least_squares, accuracy
from scripts.proj1_helpers import predict_labels

y_pred = np.zeros(len(y_test))
best_degrees = [1, 4, 3, 4]

for i, model in enumerate(models):
    # Build polynomial matrix
    _phi_train = build_poly(model['x_train'], best_degrees[i])
    _phi_test = build_poly(model['x_test'], best_degrees[i])
    print(np.shape(model['x_train']), np.shape(model['x_test']))
    loss, weights = least_squares(model['y_train'], _phi_train)
    y_pred_tmp = predict_labels(weights, _phi_test)
    y_pred[model['id_test']] = y_pred_tmp
    print(accuracy(model['y_train'], _phi_train.dot(weights)))

In [ ]:
from scripts.proj1_helpers import create_csv_submission

create_csv_submission(ids, y_pred, 'submission.csv')
print('Results saved ...')

# 3. Model 
..... meeeeehhh

In [ ]:
from scripts.implementations import build_poly, least_squares, least_squares_GD, accuracy

xt =  build_poly(x, 3)
loss, w = least_squares(yb, xt)
print(loss, np.shape(w))
#loss, w = least_squares_GD(yb, xt, max_iters=700, loss_name='mae')
#print(loss)

print(np.shape(x))

In [ ]:
accuracy(yb, xt.dot(w))

In [ ]:
from scripts.ml import cross_validation_ls

_acc = []
_loss_tr = []
_loss_te = []

for degree in range(1,6):
    print('Least square, deg: {}'.format(degree))
    acc, loss_tr, loss_te = cross_validation_ls(yb, x, degree=degree)
    _acc.append(acc); _loss_te.append(loss_te), _loss_tr.append(loss_tr)

In [ ]:
plt.plot(_acc)

In [ ]:
from scripts.ml import cross_validation_ridge

_acc = []
_loss_tr = []
_loss_te = []

for degree in range(1,6):
    print('Ridge, deg: {}'.format(degree))
    acc, loss_tr, loss_te = cross_validation_ridge(yb, x, degree=degree)
    _acc.append(acc); _loss_te.append(loss_te), _loss_tr.append(loss_tr)

In [ ]:
plt.plot(_acc)